In [48]:
import os
import sys
import numpy as np
import pandas as pd
import datetime as dt
import yaml
with open("/Users/peteraltamura/Documents/GitHub/liteSaber/data/configuration.yaml", "rb") as c:
    CONFIG = yaml.load(c)
pd.options.display.max_columns = None

/anaconda2/envs/litesaber_jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [49]:
# Parameters
dd = dt.datetime.now().strftime("%Y_%m_%d")
print(dd)

2019_09_19


# Assumptions
- We will calculate this for top 50 batters every day
- We have the starters for today

# Stats Sets We Need
- batter recent performance
    - batter_saber_team.parquet
- batter performance against pitcher
    - innings.parquet
- batter performance against opponent bullpen
    - innings.parquet
- starter
    - innings.parquet
    - preview_pull
- starter recent performance
    - pitcher_saber_team.parquet
- starter avg innings per game
    - pitcher_saber_team.parquet
    - innings.parquet
    - pitching.parquet
- target
    - innings.parquet

# Batter Recent Performance

In [50]:
path = CONFIG.get('paths').get('batter_saber')
dd_list = [p for p in os.listdir(path)]
df_bat_perf = pd.concat(
    objs=[pd.read_parquet(path+"{}/".format(dd)+'batter_saber_team.parquet') for dd in dd_list 
          if "batter_saber_team.parquet" in os.listdir(path+"{}/".format(dd))],
    axis=0
)
df_bat_perf['flag'] = pd.isnull(df_bat_perf).any(axis=1)
df_bat_perf = df_bat_perf.loc[df_bat_perf['flag'] == False, :]
df_bat_perf.drop(labels=['flag'], axis=1, inplace=True)

In [51]:
df_bat_perf = df_bat_perf.loc[:, [
    'gameId', 'gameDate', 'team', 'batterId', 
    'batterWalkPercentage_trail3',
    'batterWalkPercentage_trail6', 'batterKPercentage_trail3',
    'batterKPercentage_trail6', 'batterISO_trail3', 'batterISO_trail6',
    'batterBABIP_trail3', 'batterBABIP_trail6', 'woba_trail3',
    'woba_trail6', 'ab_trail3', 'ab_trail6'
]]
df_bat_perf.head(10)

,gameId,gameDate,team,batterId,batterWalkPercentage_trail3,batterWalkPercentage_trail6,batterKPercentage_trail3,batterKPercentage_trail6,batterISO_trail3,batterISO_trail6,batterBABIP_trail3,batterBABIP_trail6,woba_trail3,woba_trail6,ab_trail3,ab_trail6
13,gid_2018_08_01_anamlb_tbamlb_1/,2018-08-01,tba,592773,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,2.0
14,gid_2018_08_01_anamlb_tbamlb_1/,2018-08-01,tba,489265,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,2.0
15,gid_2018_08_01_anamlb_tbamlb_1/,2018-08-01,tba,621237,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,2.0
33,gid_2018_08_01_balmlb_nyamlb_1/,2018-08-01,nya,544369,0.090909,0.045455,0.181818,0.181818,0.115000,0.108500,0.111111,0.388889,0.131500,0.333565,11.0,22.0
34,gid_2018_08_01_balmlb_nyamlb_1/,2018-08-01,nya,543305,0.000000,0.045455,0.166667,0.181818,0.093000,0.121333,0.100000,0.277778,0.074000,0.256348,12.0,22.0
35,gid_2018_08_01_balmlb_nyamlb_1/,2018-08-01,nya,650402,0.090909,0.095238,0.000000,0.142857,0.090667,0.117833,0.363636,0.333333,0.385417,0.324957,11.0,21.0
36,gid_2018_08_01_balmlb_nyamlb_1/,2018-08-01,nya,595885,0.090909,0.090909,0.000000,0.090909,0.100667,0.107833,0.636364,0.400000,0.639333,0.385417,11.0,22.0
37,gid_2018_08_01_balmlb_nyamlb_1/,2018-08-01,nya,609280,0.083333,0.041667,0.166667,0.166667,0.119000,0.106000,0.900000,0.500000,0.756231,0.428760,12.0,24.0
38,gid_2018_08_01_balmlb_nyamlb_1/,2018-08-01,nya,519222,0.076923,0.083333,0.307692,0.166667,0.124000,0.107333,0.777778,0.550000,0.548000,0.472962,13.0,24.0
39,gid_2018_08_01_balmlb_nyamlb_1/,2018-08-01,nya,453203,0.076923,0.083333,0.384615,0.208333,0.120000,0.110333,0.625000,0.631579,0.393786,0.507115,13.0,24.0


# Batter Against Pitcher

In [52]:
path = CONFIG.get('paths').get('normalized')
years = [x.split("_")[1][:4] for x in os.listdir(path)]
years = list(set(years))
print(years)
df_inning_at = []
for yr in years:
    dd_list = [x for x in os.listdir(path) if yr in x]
    df_bm_all = pd.concat(
        objs=[pd.read_parquet(path+"{}/".format(dd)+'innings.parquet') for dd in dd_list 
              if "innings.parquet" in os.listdir(path+"{}/".format(dd))],
        axis=0
    )
    cols = ['atbat_batter', 'atbat_event', 'atbat_event_num', 
            'atbat_pitcher', 'game_id', 'gameId', 'gameDate', 
            'home_starting_pitcher', 'away_starting_pitcher',
            'inning_half']
    df_bm_all = df_bm_all.loc[:, cols]
    df_bm_all['year'] = yr
    df_bm_all = df_bm_all.drop_duplicates(inplace=False)
    df_bm_all = df_bm_all.loc[df_bm_all['gameId'].notnull(), :]
    df_inning_at.append(df_bm_all)
df_bat_mtch = pd.concat(df_inning_at, axis=0)
df_bat_mtch = df_bat_mtch.drop_duplicates(inplace=False)
df_bat_mtch.loc[:, 'hr_flag'] = (
    df_bat_mtch.atbat_event.str.replace(" ", "").\
    str.lower().str.contains('homerun').astype(float)
)

print("atbat_side")
# Add flag for if atbat batterId is home or away for game
df_bat_mtch.loc[:, 'atbat_side'] = np.NaN
df_bat_mtch.loc[df_bat_mtch['inning_half'] == 'top', 'atbat_side'] = 'away'
df_bat_mtch.loc[df_bat_mtch['inning_half'] == 'bottom', 'atbat_side'] = 'home'
#assert all(df_bat_mtch['atbat_side'].notnull())

df_bat_mtch.reset_index(drop=True, inplace=True)
print("atbat_opp_starter")
# Add atbat_opp_starter depending on what team batter is on
df_bat_mtch.loc[:, 'atbat_opp_starter'] = np.NaN
df_bat_mtch.loc[df_bat_mtch['inning_half'] == 'top', 'atbat_opp_starter'] = df_bat_mtch['home_starting_pitcher']
df_bat_mtch.loc[df_bat_mtch['inning_half'] == 'bottom', 'atbat_opp_starter'] = df_bat_mtch['away_starting_pitcher']
df_bat_mtch.shape
print('done all')

['2017', '2019', '2018']
Stacking Done
starting
done all


In [53]:
df_bat_mtch[['gameId', 'atbat_batter', 'atbat_event_num', 'hr_flag']].head(10)

,gameId,atbat_batter,atbat_event_num,hr_flag
0,gid_2017_03_13_bosmlb_tormlb_1/,571788,6,0.0
1,gid_2017_03_13_bosmlb_tormlb_1/,592710,10,0.0
2,gid_2017_03_13_bosmlb_tormlb_1/,643217,14,0.0
3,gid_2017_03_13_bosmlb_tormlb_1/,607680,19,0.0
4,gid_2017_03_13_bosmlb_tormlb_1/,453064,23,0.0
5,gid_2017_03_13_bosmlb_tormlb_1/,434778,27,0.0
6,gid_2017_03_13_bosmlb_tormlb_1/,501800,32,0.0
7,gid_2017_03_13_bosmlb_tormlb_1/,519048,38,0.0
8,gid_2017_03_13_bosmlb_tormlb_1/,518490,42,0.0
9,gid_2017_03_13_bosmlb_tormlb_1/,425834,47,0.0


# Calculations

Calc: Batter Recent Performance

In [32]:
# Recent Performance for this year

# Calc: Batter Against Pitcher This Year

In [54]:
# Atbats
df_bat_rec = df_bat_mtch.loc[df_bat_mtch['year'] == dd[:4], :]



df_bat_rec.loc[:, 'year'] = df_bat_rec['gameDate'].dt.year.astype(str)
df_bat_rec_agg = df_bat_rec.groupby(
    by=['year', 'gameId', 'atbat_batter', 'atbat_opp_starter'],
    as_index=False
).agg({'hr_flag': 'max',
       'atbat_event_num': 'count'})
df_bat_rec_agg.rename(columns={'atbat_event_num': 'atbat_count',
                               'hr_flag': 'hr_count'}, 
                     inplace=True)

# Bring to Year Level
# Sort
df_bat_rec_agg.sort_values(
    by=['year', 'gameId', 'atbat_batter', 'atbat_opp_starter'],
    ascending=True, 
    inplace=True
)
# Cumulative Sums for each year
df_bat_rec_agg['hr_count_roll'] = (
    df_bat_rec_agg.groupby(['year', 'atbat_batter', 'atbat_opp_starter'])
    ['hr_count'].cumsum().reset_index(drop=True)
)
df_bat_rec_agg['atbat_count_roll'] = (
    df_bat_rec_agg.groupby(['year', 'atbat_batter', 'atbat_opp_starter'])
    ['atbat_count'].cumsum().reset_index(drop=True)
)
df_bat_rec_agg.loc[:, 'hr_prop'] = (
    df_bat_rec_agg['hr_count_roll'] / 
    df_bat_rec_agg['atbat_count_roll']
)
df_bat_rec_agg.tail(5)

/anaconda2/envs/litesaber_jupyter/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,year,gameId,atbat_batter,atbat_opp_starter,hr_count,atbat_count,hr_count_roll,atbat_count_roll,hr_prop
50218,2019,gid_2019_09_18_wasmlb_slnmlb_1/,645302,425794,0.0,4,1.0,9,0.111111
50219,2019,gid_2019_09_18_wasmlb_slnmlb_1/,657557,453286,0.0,3,0.0,8,0.000000
50220,2019,gid_2019_09_18_wasmlb_slnmlb_1/,664056,453286,0.0,2,0.0,6,0.000000
50221,2019,gid_2019_09_18_wasmlb_slnmlb_1/,665742,425794,0.0,4,0.0,8,0.000000
50222,2019,gid_2019_09_18_wasmlb_slnmlb_1/,669242,453286,1.0,3,1.0,3,0.333333


# Calc: Batter Against Pitcher All Time

In [55]:
# Atbats
df_bat_rec_agg_alltime = df_bat_mtch.loc[df_bat_mtch['year'].astype(int) < int(dd[:4]), :]


df_bat_rec_agg_alltime = df_bat_rec_agg_alltime.groupby(
    by=['gameId', 'atbat_batter', 'atbat_opp_starter'],
    as_index=False
).agg({'hr_flag': 'max',
       'atbat_event_num': 'count'})
df_bat_rec_agg_alltime.rename(columns={'atbat_event_num': 'atbat_count',
                               'hr_flag': 'hr_count'}, 
                     inplace=True)

# Bring to Year Level
# Sort
df_bat_rec_agg_alltime.sort_values(
    by=['gameId', 'atbat_batter', 'atbat_opp_starter'],
    ascending=True, 
    inplace=True
)
# Cumulative Sums for each year
df_bat_rec_agg_alltime['hr_count_roll'] = (
    df_bat_rec_agg_alltime.groupby(['atbat_batter', 'atbat_opp_starter'])
    ['hr_count'].cumsum().reset_index(drop=True)
)
df_bat_rec_agg_alltime['atbat_count_roll'] = (
    df_bat_rec_agg_alltime.groupby(['atbat_batter', 'atbat_opp_starter'])
    ['atbat_count'].cumsum().reset_index(drop=True)
)
df_bat_rec_agg_alltime.loc[:, 'hr_prop'] = (
    df_bat_rec_agg_alltime['hr_count_roll'] / 
    df_bat_rec_agg_alltime['atbat_count_roll']
)
df_bat_rec_agg_alltime.tail(5)

,gameId,atbat_batter,atbat_opp_starter,hr_count,atbat_count,hr_count_roll,atbat_count_roll,hr_prop
103543,gid_2018_10_28_bosmlb_lanmlb_1/,621035,456034,0.0,3,0.0,7,0.0
103544,gid_2018_10_28_bosmlb_lanmlb_1/,624577,456034,0.0,3,0.0,6,0.0
103545,gid_2018_10_28_bosmlb_lanmlb_1/,641355,456034,0.0,1,0.0,2,0.0
103546,gid_2018_10_28_bosmlb_lanmlb_1/,643217,477132,0.0,4,0.0,9,0.0
103547,gid_2018_10_28_bosmlb_lanmlb_1/,646240,477132,0.0,4,0.0,7,0.0


# Calc: Batter Against Pitcher All-Time

# Batter Performance Against Opponent Bullpen This Year

# Shift Target back (so next game HR flag becomes current target)

In [61]:
# Aggregate for flag of any HR per game
target = df_bat_mtch.groupby(
    by=['atbat_batter', 'gameId', 'gameDate'],
    as_index=False
).agg({'hr_flag': 'max'})

# Shift back gameId to make the HR count reflect the game that followed
target.loc[:, 'gameId_back'] = np.NaN
target.loc[(target['atbat_batter'] == target['atbat_batter'].shift(1)), 'gameId_back'] = target['gameId'].shift(1)
target.rename(columns={'hr_flag': 'hr_flag_next_game'})
target.drop(labels=['gameId', 'gameDate'], axis=1, inplace=True)

# Combine Features

In [64]:
# Add starter to batter recent agg
df_ft = df_bat_mtch.loc[:, ['gameId', 'gameDate', 'atbat_batter', 'atbat_opp_starter']].drop_duplicates(inplace=False)
df_ft.sort_values(by=['gameId', 'atbat_batter', 'atbat_opp_starter'], ascending=True, inplace=True)
df_ft.drop_duplicates(subset=['gameId', 'atbat_batter'], inplace=True)

df_ft.sort_values(by=['gameDate'], ascending=True, inplace=True)
df_ft = pd.merge(
    df_ft,
    df_bat_perf,
    how='inner',
    left_on=['gameId', 'gameDate', 'atbat_batter'],
    right_on=['gameId', 'gameDate', 'batterId'],
    validate='1:1'
)
df_ft.drop(labels=['batterId'], axis=1, inplace=True)
#df_bat_rec_agg.rename(
#    columns={x: x+"_mtchup_starter" for x in df_bat_rec_agg.columns 
#             if x not in ['gameId', 'atbat_batter', 'atbat_opp_starter', 'year']},
#    inplace=True
#)
df_ft = pd.merge(
    df_ft,
    df_bat_rec_agg,
    how='inner',
    left_on=['gameId', 'atbat_batter', 'atbat_opp_starter'],
    right_on=['gameId', 'atbat_batter', 'atbat_opp_starter'],
    validate='1:1'
)
print(df_ft.shape)

# Merge on Target
df_ft = pd.merge(
    df_ft,
    target,
    how='inner',
    left_on=['gameId', 'atbat_batter'],
    right_on=['gameId_back', 'atbat_batter'],
    validate='1:1'
)
df_ft.drop(labels=['gameId_back'], axis=1, inplace=True)
print(df_ft.shape)

(33806, 23)
(32792, 24)


In [65]:
df_ft.tail()

,gameId,gameDate,atbat_batter,atbat_opp_starter,team,batterWalkPercentage_trail3,batterWalkPercentage_trail6,batterKPercentage_trail3,batterKPercentage_trail6,batterISO_trail3,batterISO_trail6,batterBABIP_trail3,batterBABIP_trail6,woba_trail3,woba_trail6,ab_trail3,ab_trail6,year_mtchup_starter,hr_count_mtchup_starter,atbat_count_mtchup_starter,hr_count_roll_mtchup_starter,atbat_count_roll_mtchup_starter,hr_prop_mtchup_starter,hr_flag
32787,gid_2019_09_17_kcamlb_oakmlb_1/,2019-09-17,501981,605347,oak,0.200000,0.166667,0.200000,0.125000,0.190000,0.191500,0.250000,0.450000,0.263000,0.518107,10.0,24.0,2019,0.0,3,0.0,3,0.0,0.0
32788,gid_2019_09_17_kcamlb_oakmlb_1/,2019-09-17,460086,474463,kca,0.000000,0.136364,0.083333,0.181818,0.197333,0.196167,0.500000,0.312500,0.682917,0.530160,12.0,22.0,2019,0.0,4,0.0,4,0.0,0.0
32789,gid_2019_09_17_detmlb_clemlb_1/,2019-09-17,656185,643617,cle,0.100000,0.045455,0.200000,0.090909,0.197333,0.197833,0.750000,0.578947,0.616727,0.634609,10.0,22.0,2019,0.0,5,0.0,5,0.0,0.0
32790,gid_2019_09_17_kcamlb_oakmlb_1/,2019-09-17,656305,605347,oak,0.153846,0.125000,0.153846,0.083333,0.191000,0.191000,0.400000,0.428571,0.553600,0.497556,13.0,24.0,2019,0.0,4,0.0,4,0.0,0.0
32791,gid_2019_09_17_kcamlb_oakmlb_1/,2019-09-17,657656,605347,oak,0.083333,0.125000,0.083333,0.083333,0.193333,0.189333,0.444444,0.400000,0.845077,0.589556,12.0,24.0,2019,0.0,1,0.0,1,0.0,0.0


In [77]:
if not os.path.exists("/Volumes/Samsung_T5/mlb/gdApi/99_hr_prop_ft/"):
    os.makedirs("/Volumes/Samsung_T5/mlb/gdApi/99_hr_prop_ft/")
df_ft.to_parquet('/Volumes/Samsung_T5/mlb/gdApi/99_hr_prop_ft/featurespace_all.parquet')

ModuleNotFoundError: No module named 'sklearn'